In [6]:
import scipy
import pandas as pd
import numpy as np


In [20]:
#initial solution for given A \in R^(m,n) and b \in R^m, if the index set of an initial basis is given then it will be used.
def initial_solution2(A,b, init_basis=None):
    if np.linalg.matrix_rank(np.c_[A, b]) != np.linalg.matrix_rank(A):
        return 'Ax=b is not solvable!'
    A_hat = A
    b_hat = b
    in_the_basis = range(A.shape[1],A.shape[1]+A.shape[0])
    in_the_basis = np.array(in_the_basis)
    kakao = (in_the_basis>=A.shape[1])
    if init_basis == None:
        for i in np.where(kakao)[0]:
            if (np.all(A_hat[i]==0)) &(b_hat[i]==0):
                A_hat = np.delete(A_hat, i,0)
                b_hat = np.delete(b_hat,i)
                in_the_basis = np.delete(in_the_basis,i)
            else:

                ind = (A_hat[i]!=0).argmax(axis=0)
                A_hat, b_hat = pivot(A_hat,b_hat,i,ind)
                in_the_basis[i]=ind
    if init_basis!=None:
        init_basis = list(init_basis)
        for i in np.where(kakao)[0]:
            if (np.all(A_hat[i]==0)) &(b_hat[i]==0):
                A_hat = np.delete(A_hat, i,0)
                b_hat = np.delete(b_hat,i)
                in_the_basis = np.delete(in_the_basis,i)
            else:
                ind = [j for j in init_basis if A_hat[i][j]!=0][0]
                init_basis.remove(ind)
                A_hat, b_hat = pivot(A_hat,b_hat,i,ind)
                in_the_basis[i]=ind
    return(in_the_basis, A_hat, b_hat)


In [21]:
# pivot on given index1,index2 and table A|b
def pivot(A, b,index1, index2):
    P = np.identity(A.shape[0])
    v= -A[:, index2]/A[index1, index2]
    v[index1] = 1/A[index1, index2]
    P[:, index1] = v
    new_A = np.matmul(P, A)
    new_b_hat = np.matmul(P, b)
    return(new_A, new_b_hat)
    
    

In [9]:
#creates a pandas dataframe of the solution
def nice_table(table,b_hat=None,init_basis=None):
    tableau = pd.DataFrame(table, columns = ['x'+str(i) for i in range(table.shape[1])])
    tableau['b'] = b_hat
    if np.all(init_basis) != None:
        tableau.index = ['x'+str(i) for i in init_basis]
    return(tableau)
    

In [10]:
#runs the MBU-simplex algorithm for given A \in R^(m,n) and b \in R^m, if the index set of an initial basis is given then it will be used.
def MBU_simplex(A,b, init_basis = None):
    #initial solution
    init_basis, table, b_hat = initial_solution2(A,b, init_basis)
    print('initial solution:', '\n' ,nice_table(table, b_hat, init_basis))
    end_while= np.all(np.array(b_hat)>=0)
    no_print=False
    countr_outer = 0
    while end_while==False:
        countr_outer+=1
        #set of indices where b_hat<0
        index_minus_orig  = np.array(init_basis)[np.where(np.array(b_hat)<0)]
        index_minus = np.array(range(A.shape[0]))[np.where(np.array(b_hat)<0)]
        r_original = np.min(index_minus_orig)
        r = index_minus[np.argmin(index_minus_orig)]
        rDone = False
        countr_inner = 0
        print('r=', 'x'+str(r_original))
        while rDone==False:
            
            if list(np.where(table[r, :]<0)[0])==[]:
                print('no feasible solution!')
                no_print = True
                rDone=True
                end_while=True
            else:
                J_r_=np.where(table[r, :]<0)
                J_r_original = np.array(range(table.shape[1]))[J_r_]
                s = np.min(J_r_)
                s_original = range(table.shape[1])[s]
                theta1 = b_hat[r]/table[r,s]
                print('s=', 'x'+str(s_original))
                good_row = table[:,s][np.where((table[:,s]>0) & (b_hat>=0))]
                good_b = b_hat[np.where((table[:,s]>0) & (b_hat>=0))]
                good_index = np.where((table[:,s]>0) & (b_hat>=0))
                if list(good_b)==[]:
                    theta2 = np.inf
                else:
                    theta2 = np.min(good_b/good_row)
                    q = good_index[0][np.argmin(good_b/good_row)]
                    q_orig = init_basis[q]
                print('theta2 =', theta2, ', theta1 =', theta1)
                
                if theta1<=theta2:
                    print('theta2>= theta1 => pivot on (r,s)=', '(','x'+str(r_original),',','x'+str(s_original), ')')
                    table, b_hat = pivot(np.array(table), np.array(b_hat), r,s)
                    rDone=True
                    init_basis[r] = s_original
                    print('new basis: ', ['x'+str(i) for i in init_basis])
                    countr_inner +=1
                    print('iterations: outer:', str(countr_outer), ' inner:', str(countr_inner))
                    print('_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ')
                if theta2<theta1:
                    print('theta2< theta1 => pivot on (q,s)=', '(','x'+str(q_orig),',','x'+str(s_original), ')')
                    table, b_hat = pivot(np.array(table), np.array(b_hat), q,s)
                    init_basis[q] = s_original
                    print('new basis: ', ['x'+str(i) for i in init_basis])
                    countr_inner +=1
                    print('iterations: outer:', str(countr_outer), ' inner:', str(countr_inner))
                    print('. . . . . . . . . . . . . . . . . . . . . . .')
                print(nice_table(table, b_hat, init_basis))
                end_while = np.all(np.array(b_hat)>=0)
    if no_print == False:
        solution = np.zeros((1,A.shape[1]))[0]
        solution[init_basis]=b_hat
        print('______________________________________________')
        print('x =',solution,'\n', 'check:', '\n',  nice_table(A, b), '\n','A*x = ', np.matmul(A, solution))
        return(solution)
    else:
        return('no feasible solution!')
                    
                    
    

# examples

## first example

In [11]:
A = np.array([[1,0,0], [0,1,0], [0,0,-1]])
b=np.array([1,2,3])

In [12]:
MBU_simplex(A,b)

initial solution: 
      x0   x1   x2    b
x0  1.0  0.0  0.0  1.0
x1  0.0  1.0  0.0  2.0
x2  0.0  0.0  1.0 -3.0
r= x2
no feasible solution!


'no feasible solution!'

## second example

In [13]:
A=np.array([[1,2,0,-1,1,1,0],[1,0,2,1,1,0,0], [0,2,0,0,1,0,-1],[0,2,0,0,1,0,-1]])
b = np.array([30,80,40, 40])

In [14]:
MBU_simplex(A,b)

initial solution: 
      x0   x1   x2   x3   x4   x5   x6     b
x0  1.0  0.0  0.0 -1.0  0.0  1.0  1.0 -10.0
x1  0.0  1.0  0.0  0.0  0.5  0.0 -0.5  20.0
x2  0.0  0.0  1.0  1.0  0.5 -0.5 -0.5  45.0
r= x0
s= x3
theta2 = 45.0 , theta1 = 10.0
theta2>= theta1 => pivot on (r,s)= ( x0 , x3 )
new basis:  ['x3', 'x1', 'x2']
iterations: outer: 1  inner: 1
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
     x0   x1   x2   x3   x4   x5   x6     b
x3 -1.0  0.0  0.0  1.0  0.0 -1.0 -1.0  10.0
x1  0.0  1.0  0.0  0.0  0.5  0.0 -0.5  20.0
x2  1.0  0.0  1.0  0.0  0.5  0.5  0.5  35.0
______________________________________________
x = [ 0. 20. 35. 10.  0.  0.  0.] 
 check: 
    x0  x1  x2  x3  x4  x5  x6   b
0   1   2   0  -1   1   1   0  30
1   1   0   2   1   1   0   0  80
2   0   2   0   0   1   0  -1  40
3   0   2   0   0   1   0  -1  40 
 A*x =  [30. 80. 40. 40.]


array([ 0., 20., 35., 10.,  0.,  0.,  0.])

### with given initial solution

In [15]:
MBU_simplex(A,b, [5,2,6])

initial solution: 
      x0   x1   x2   x3   x4   x5   x6     b
x5  1.0  2.0  0.0 -1.0  1.0  1.0  0.0  30.0
x2  0.5  0.0  1.0  0.5  0.5  0.0  0.0  40.0
x6  0.0 -2.0  0.0  0.0 -1.0  0.0  1.0 -40.0
r= x6
s= x1
theta2 = 15.0 , theta1 = 20.0
theta2< theta1 => pivot on (q,s)= ( x5 , x1 )
new basis:  ['x1', 'x2', 'x6']
iterations: outer: 1  inner: 1
. . . . . . . . . . . . . . . . . . . . . . .
     x0   x1   x2   x3   x4   x5   x6     b
x1  0.5  1.0  0.0 -0.5  0.5  0.5  0.0  15.0
x2  0.5  0.0  1.0  0.5  0.5  0.0  0.0  40.0
x6  1.0  0.0  0.0 -1.0  0.0  1.0  1.0 -10.0
s= x3
theta2 = 80.0 , theta1 = 10.0
theta2>= theta1 => pivot on (r,s)= ( x6 , x3 )
new basis:  ['x1', 'x2', 'x3']
iterations: outer: 1  inner: 2
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
     x0   x1   x2   x3   x4   x5   x6     b
x1  0.0  1.0  0.0  0.0  0.5  0.0 -0.5  20.0
x2  1.0  0.0  1.0  0.0  0.5  0.5  0.5  35.0
x3 -1.0  0.0  0.0  1.0  0.0 -1.0 -1.0  10.0
______________________________________________
x = [ 0. 20. 35. 

array([ 0., 20., 35., 10.,  0.,  0.,  0.])

## third example

In [16]:
A=np.array([[-1,0,-1,1,1,0,0], [-1, -1, -2, -3, 0, 1, 0], [1, 0, 0, -2, 0, 0, 1]])
b=np.array([-2,-2, -3])

In [17]:
MBU_simplex(A,b)

initial solution: 
      x0   x1   x2   x3   x4   x5   x6    b
x0  1.0  0.0  0.0 -2.0  0.0  0.0  1.0 -3.0
x1  0.0  1.0  0.0  3.0  2.0 -1.0  1.0 -5.0
x2  0.0  0.0  1.0  1.0 -1.0  0.0 -1.0  5.0
r= x0
s= x3
theta2 = 5.0 , theta1 = 1.5
theta2>= theta1 => pivot on (r,s)= ( x0 , x3 )
new basis:  ['x3', 'x1', 'x2']
iterations: outer: 1  inner: 1
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
     x0   x1   x2   x3   x4   x5   x6    b
x3 -0.5  0.0  0.0  1.0  0.0  0.0 -0.5  1.5
x1  1.5  1.0  0.0  0.0  2.0 -1.0  2.5 -9.5
x2  0.5  0.0  1.0  0.0 -1.0  0.0 -0.5  3.5
r= x1
s= x5
theta2 = inf , theta1 = 9.5
theta2>= theta1 => pivot on (r,s)= ( x1 , x5 )
new basis:  ['x3', 'x5', 'x2']
iterations: outer: 2  inner: 1
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
     x0   x1   x2   x3   x4   x5   x6    b
x3 -0.5  0.0  0.0  1.0  0.0  0.0 -0.5  1.5
x5 -1.5 -1.0  0.0  0.0 -2.0  1.0 -2.5  9.5
x2  0.5  0.0  1.0  0.0 -1.0  0.0 -0.5  3.5
______________________________________________
x = [0.  0.  3.5 1.5 0.  

array([0. , 0. , 3.5, 1.5, 0. , 9.5, 0. ])

In [18]:
A=np.array([[1,-2,1,0,0], [0,0,0,1,0], [-4,0,0,0,1]])
b=np.array([0,1,-2])

In [19]:
MBU_simplex(A,b, [2,3,4])

initial solution: 
      x0   x1   x2   x3   x4    b
x2  1.0 -2.0  1.0  0.0  0.0  0.0
x3  0.0  0.0  0.0  1.0  0.0  1.0
x4 -4.0  0.0  0.0  0.0  1.0 -2.0
r= x4
s= x0
theta2 = 0.0 , theta1 = 0.5
theta2< theta1 => pivot on (q,s)= ( x2 , x0 )
new basis:  ['x0', 'x3', 'x4']
iterations: outer: 1  inner: 1
. . . . . . . . . . . . . . . . . . . . . . .
     x0   x1   x2   x3   x4    b
x0  1.0 -2.0  1.0  0.0  0.0  0.0
x3  0.0  0.0  0.0  1.0  0.0  1.0
x4  0.0 -8.0  4.0  0.0  1.0 -2.0
s= x1
theta2 = inf , theta1 = 0.25
theta2>= theta1 => pivot on (r,s)= ( x4 , x1 )
new basis:  ['x0', 'x3', 'x1']
iterations: outer: 1  inner: 2
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
     x0   x1   x2   x3     x4     b
x0  1.0  0.0  0.0  0.0 -0.250  0.50
x3  0.0  0.0  0.0  1.0  0.000  1.00
x1  0.0  1.0 -0.5  0.0 -0.125  0.25
______________________________________________
x = [0.5  0.25 0.   1.   0.  ] 
 check: 
    x0  x1  x2  x3  x4  b
0   1  -2   1   0   0  0
1   0   0   0   1   0  1
2  -4   0   0   0   1 -

array([0.5 , 0.25, 0.  , 1.  , 0.  ])

______________________________________________________